In [169]:
# import the needed modules
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as req
import numpy as np

In [170]:
#request with Requests library, using Postman interceptor - empty IS-Academia form
r = req.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')

In [171]:
# define BeautifulSoup object to parse the html doc
soup = bs(r.text,'lxml')

# Parsing the HMTL form

In the following the parameters section, academic year, academic cycle and season are parsed, so that each variable is linked to its corresponding numeric code through a dictionary. Thereafter, the parameters names are obtained as the IS_Academia form accepts them.

In [184]:
# parse the section Informatique
section = {'Informatique' : soup.find('option',string='Informatique')['value']}
section

{'Informatique': '249847'}

In [185]:
# parse the academic year - loop over the years and find the code corresponding to the year string
academic_year = {}
for year in range(2007,2017):
    period = str(year)+'-'+str(year+1)
    academic_year[period] = soup.find('option', string = period)['value']
academic_year

{'2007-2008': '978181',
 '2008-2009': '978187',
 '2009-2010': '978195',
 '2010-2011': '39486325',
 '2011-2012': '123455150',
 '2012-2013': '123456101',
 '2013-2014': '213637754',
 '2014-2015': '213637922',
 '2015-2016': '213638028',
 '2016-2017': '355925344'}

In [186]:
# parse the academic cycle - define the sought cycles and look for the corresponding codes, as above
bachelor = []
master = []
for j in range(1,7):
    bachelor.append('Bachelor semestre '+str(j))
for j in range(1,5):
    master.append('Master semestre '+str(j))
bachelor_master = bachelor+master

academic_cycle = {}
for cycle in bachelor_master:
    academic_cycle[cycle] = soup.find('option', string = cycle)['value']
academic_cycle

{'Bachelor semestre 1': '249108',
 'Bachelor semestre 2': '249114',
 'Bachelor semestre 3': '942155',
 'Bachelor semestre 4': '942163',
 'Bachelor semestre 5': '942120',
 'Bachelor semestre 6': '942175',
 'Master semestre 1': '2230106',
 'Master semestre 2': '942192',
 'Master semestre 3': '2230128',
 'Master semestre 4': '2230140'}

In [187]:
# parse the academic season - same procedure as above
season = {}
season["Semestre d'automne"]=soup.find('option', string = "Semestre d'automne")['value']
season['Semestre de printemps']=soup.find('option', string = 'Semestre de printemps')['value']
season

{"Semestre d'automne": '2936286', 'Semestre de printemps': '2936295'}

In [188]:
# get the variables' names to fill in the IS-Academia form
variables = []
args = soup.find_all('select')
for var in args:
    variables.append(var['name'])
variables

['ww_x_UNITE_ACAD',
 'ww_x_PERIODE_ACAD',
 'ww_x_PERIODE_PEDAGO',
 'ww_x_HIVERETE']